In [8]:
import requests
from datetime import date
import pandas as pd

In [9]:
BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"

def _fmt_mmddyyyy(d: date) -> str:
    return d.strftime("%m-%d-%Y")

def cotacao_dolar_periodo_df(data_ini: date, data_fim: date) -> pd.DataFrame:
    url = f"{BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    params = {
        "@dataInicial": f"'{_fmt_mmddyyyy(data_ini)}'",
        "@dataFinalCotacao": f"'{_fmt_mmddyyyy(data_fim)}'",
        "$format": "json",
        "$select": "cotacaoCompra,cotacaoVenda,dataHoraCotacao",
        "$top": 10000,  # período pode retornar muitas linhas (vários boletins por dia)
    }

    r = requests.get(url, params=params, timeout=30)
    # Se der erro, ajuda a depurar vendo o payload retornado
    if not r.ok:
        raise requests.HTTPError(f"{r.status_code} - {r.text}", response=r)

    payload = r.json()
    df = pd.DataFrame(payload.get("value", []))

    if df.empty:
        return df

    df["dataHoraCotacao"] = pd.to_datetime(df["dataHoraCotacao"])
    df = df.sort_values("dataHoraCotacao").reset_index(drop=True)
    return df

In [10]:
df_periodo = cotacao_dolar_periodo_df(date(2026, 1, 1), date(2026, 2, 5))
print(df_periodo)

    cotacaoCompra  cotacaoVenda         dataHoraCotacao
0          5.4366        5.4372 2026-01-02 13:03:33.254
1          5.4345        5.4351 2026-01-05 13:10:14.896
2          5.3791        5.3797 2026-01-06 13:12:07.358
3          5.3874        5.3880 2026-01-07 13:06:27.001
4          5.3854        5.3860 2026-01-08 13:04:28.616
5          5.3701        5.3707 2026-01-09 13:09:29.054
6          5.3754        5.3760 2026-01-12 13:08:29.056
7          5.3758        5.3764 2026-01-13 13:08:54.165
8          5.3789        5.3795 2026-01-14 13:04:32.411
9          5.3840        5.3846 2026-01-15 13:05:25.660
10         5.3792        5.3798 2026-01-16 13:04:26.486
11         5.3647        5.3653 2026-01-19 13:07:27.662
12         5.3784        5.3790 2026-01-20 13:02:30.787
13         5.3362        5.3368 2026-01-21 13:04:32.866
14         5.3112        5.3118 2026-01-22 13:07:26.401
15         5.2872        5.2879 2026-01-23 13:09:25.549
16         5.2754        5.2760 2026-01-26 13:09

In [ ]:
codigo = """
import streamlit as st
import pandas as pd
import requests
from datetime import date
import plotly.graph_objects as go


BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"

def _fmt_mmddyyyy(d: date) -> str:
    return d.strftime("%m-%d-%Y")

@st.cache_data(ttl=3600)
def cotacao_dolar_periodo_df(data_ini: date, data_fim: date) -> pd.DataFrame:
    url = f"{BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    params = {
        "@dataInicial": f"'{_fmt_mmddyyyy(data_ini)}'",
        "@dataFinalCotacao": f"'{_fmt_mmddyyyy(data_fim)}'",
        "$format": "json",
        "$select": "cotacaoCompra,cotacaoVenda,dataHoraCotacao",
        "$top": 10000,
    }

    r = requests.get(url, params=params, timeout=30)

    if not r.ok:
        raise requests.HTTPError(f"{r.status_code} - {r.text}", response=r)

    payload = r.json()
    df = pd.DataFrame(payload.get("value", []))

    if df.empty:
        return df

    df["dataHoraCotacao"] = pd.to_datetime(df["dataHoraCotacao"])
    df = df.sort_values("dataHoraCotacao").reset_index(drop=True)

    return df


# ---------------- UI ----------------

st.set_page_config(
    page_title="PTAX - Cotação do Dólar",
    layout="wide"
)

st.title("💵 PTAX — Cotação do Dólar (Banco Central)")
st.write("Consulta automática via API oficial do BCB.")

with st.sidebar:
    st.header("Parâmetros")

    hoje = date.today()

    data_ini = st.date_input(
        "Data inicial",
        value=date(hoje.year, 1, 1)
    )

    data_fim = st.date_input(
        "Data final",
        value=hoje
    )

    consultar = st.button("Consultar")


if consultar:

    if data_ini > data_fim:
        st.error("A data inicial não pode ser maior que a data final.")
        st.stop()

    with st.spinner("Consultando API do Banco Central..."):
        try:
            df = cotacao_dolar_periodo_df(data_ini, data_fim)
        except Exception as e:
            st.error(f"Erro ao consultar API: {e}")
            st.stop()

    if df.empty:
        st.warning("Nenhuma cotação encontrada para o período.")
        st.stop()

    # ---------- Métrica principal ----------
    ultima = df.iloc[-1]

    col1, col2 = st.columns(2)

    col1.metric(
        "Última cotação de compra",
        f"R$ {ultima['cotacaoCompra']:.4f}"
    )

    col2.metric(
        "Última cotação de venda",
        f"R$ {ultima['cotacaoVenda']:.4f}"
    )

    # ---------- Gráfico ----------
    st.subheader("📈 Evolução do dólar (Plotly)")

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df["dataHoraCotacao"],
        y=df["cotacaoCompra"],
        mode="lines",
        name="Compra"
    ))

    fig.add_trace(go.Scatter(
        x=df["dataHoraCotacao"],
        y=df["cotacaoVenda"],
        mode="lines",
        name="Venda"
    ))

    fig.update_layout(
        height=520,
        margin=dict(l=10, r=10, t=40, b=10),
        xaxis_title="Data/Hora",
        yaxis_title="R$",
    )

    # Range slider pra navegar no tempo
    fig.update_xaxes(rangeslider_visible=True)

    st.plotly_chart(fig, use_container_width=True)
    # ---------- Tabela ----------
    st.subheader("📊 Dados completos")

    st.dataframe(
        df,
        use_container_width=True
    )

    # ---------- Download ----------
    st.download_button(
        "⬇️ Baixar CSV",
        data=df.to_csv(index=False).encode("utf-8"),
        file_name="cotacao_dolar_ptax.csv",
        mime="text/csv",
    )

else:
    st.info("Selecione o período e clique em **Consultar**.")
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(codigo)

print("✅ app.py criado com sucesso!")

✅ app.py criado com sucesso!
